In [36]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from crewai import LLM

llm = LLM( model="gemini/gemini-1.5-flash",
           temperature=0.1)

llm.call("What is the capital of France?")

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {                         │
│    "error": {                                                                                                   │
│      "code": 503,                                                                                               │
│      "message": "The model is overloaded. Please try again later.",                                             │
│      "status": "UNAVAILABLE"                                                                                    │
│    }                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

InternalServerError: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The model is overloaded. Please try again later.",
    "status": "UNAVAILABLE"
  }
}


In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
from crewai import Agent

# Your existing LLM setup
# llm = LLM(model="gemini/gemini-1.5-flash", temperature=0.1)

# Define the first agent: OCR Parser
ocr_parser_agent = Agent(
    role="An OCR specialist expert in extracting text from medical documents.",
    goal="Accurately and reliably extract all text from a given medical report image.",
    backstory="You are an expert at using Optical Character Recognition (OCR) technology. You know how to handle complex layouts and medical terminology to produce clean, usable text.",
    llm=llm, # Use the LLM you've already configured
    verbose=True,
    allow_delegation=False
)
from crewai import Task

# Define the task for the OCR Parser
ocr_task = Task(
    description="Use the 'OCR Tool' to extract all text from the blood report image. The report is located at './reports/blood_report.png'.",
    agent=ocr_parser_agent,
    expected_output="A single string containing the raw text of the medical report."
)

In [ ]:
from crewai import Agent

# Define the second agent: Health Data Interpreter
health_interpreter_agent = Agent(
    role="A highly knowledgeable medical professional who interprets blood test results.",
    goal="Analyze blood report text, identify all key health indicators, and interpret their values in a simple way.",
    backstory="You are a seasoned health professional with a deep understanding of medical terminology and standard lab ranges. Your expertise lies in translating complex medical results into clear, actionable insights for non-medical professionals.",
    llm=llm,
    verbose=True,
    allow_delegation=False
)
from crewai import Task

# Define the task for the Health Data Interpreter
interpretation_task = Task(
    description="Analyze the raw text from the previous task. Identify all blood test parameters and values, and interpret whether each value is 'Normal', 'High', or 'Low'. Provide a simple explanation for each.",
    agent=health_interpreter_agent,
    context=[ocr_task], # This is how the output of the first task is passed to the second!
    expected_output="A structured JSON object with each parameter, its value, status (high/low/normal), and a simple explanation."
)

In [ ]:
from crewai import Agent, Task

# Define the third agent: Summary Generator
summary_generator_agent = Agent(
    role="A professional medical writer and communicator.",
    goal="Take the structured health data analysis and generate a comprehensive, patient-friendly summary report.",
    backstory="You are an excellent writer who can take complex medical information and transform it into a clear, concise, and helpful report for patients.",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

# Define the task for the Summary Generator
summarization_task = Task(
    description="Generate a simple, patient-friendly summary report based on the interpreted data. Highlight the most important findings and explain them in plain language.",
    agent=summary_generator_agent,
    context=[interpretation_task], # This passes the interpreted data to the summarizer
    expected_output="A well-formatted, easy-to-read report in Markdown format with key findings and explanations."
)

In [ ]:
from crewai import Crew, Process

# Create the crew with all agents and tasks
medical_report_crew = Crew(
    agents=[ocr_parser_agent, health_interpreter_agent, summary_generator_agent],
    tasks=[ocr_task, interpretation_task, summarization_task],
    process=Process.sequential,
    verbose=True # Corrected: verbose should be a boolean (True or False)
)

# Run the crew and get the final result
print("## Crew starting up...")
result = medical_report_crew.kickoff(inputs={
    'report_file_path': 'C:\Users\91990\crewai_proj\Sample reports\cbc-with-ESR-test-report-format-example-sample-template-drlogy-lab-report.webp'
})

print("\n\n################################################")
print("## Final Report Generated by the Crew:")
print("################################################")
print(result)

## Crew starting up...

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 246f289d-09da-41c8-a33d-8edcaf59d1d9                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: An OCR specialist expert in extracting text from medical documents.                                     │
│                                                                                                                 │
│  Task: Use the 'OCR Tool' to extract all text from the blood report image. The report is located at             │
│  './reports/blood_report.png'.                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: An OCR specialist expert in extracting text from medical documents.                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  I cannot access local files, including the image file './reports/blood_report.png'. Therefore, I cannot        │
│  perform OCR on the provided image and return the extracted text.  To get the desired output, please provide    │
│  the image data directly within the prompt.                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8816411c-bf7c-4ceb-b4c0-5ced6dd6bcdf                                                                     │
│  Agent: An OCR specialist expert in extracting text from medical documents.                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: A professional medical writer and communicator.                                                         │
│                                                                                                                 │
│  Task: Generate a simple, patient-friendly summary report based on the interpreted data. Highlight the most     │
│  important findings and explain them in plain language.                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: A professional medical writer and communicator.                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Blood Report Summary:  Unable to Generate Report                                                             │
│                                                                                                                 │
│  Unfortunately, I am unable to provide a blood report summary at this time.  No blood test data was provided.   │
│  To generate a helpful and accurate report, I require the specific results from your blood test, including the  │
│  values obtained for each test performed and the units of measurement used (e.g., mg/dL, mmol/L, etc.).         │
│                                                                                                                 │
│  Once you provide the blood report data, I can analyze the results and create a clear, concise, and             │
│  patient-friendly summary that explains the key findings in plain language.  This summary will highlight any    │
│  significant abnormalities and explain their potential implications.  I will also avoid using complex medical   │
│  jargon and ensure the report is easy to understand.                                                            │
│                                                                                                                 │
│  Please provide the necessary data so I can assist you.                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3e59f4f9-68f6-4915-8edb-c9cb1f514da0                                                                     │
│  Agent: A professional medical writer and communicator.                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 246f289d-09da-41c8-a33d-8edcaf59d1d9                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Blood Report Summary:  Unable to Generate Report                                               │
│                                                                                                                 │
│  Unfortunately, I am unable to provide a blood report summary at this time.  No blood test data was provided.   │
│  To generate a helpful and accurate report, I require the specific results from your blood test, including the  │
│  values obtained for each test performed and the units of measurement used (e.g., mg/dL, mmol/L, etc.).         │
│                                                                                                                 │
│  Once you provide the blood report data, I can analyze the results and create a clear, concise, and             │
│  patient-friendly summary that explains the key findings in plain language.  This summary will highlight any    │
│  significant abnormalities and explain their potential implications.  I will also avoid using complex medical   │
│  jargon and ensure the report is easy to understand.                                                            │
│                                                                                                                 │
│  Please provide the necessary data so I can assist you.                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



################################################
## Final Report Generated by the Crew:
################################################
# Blood Report Summary:  Unable to Generate Report

Unfortunately, I am unable to provide a blood report summary at this time.  No blood test data was provided.  To generate a helpful and accurate report, I require the specific results from your blood test, including the values obtained for each test performed and the units of measurement used (e.g., mg/dL, mmol/L, etc.).

Once you provide the blood report data, I can analyze the results and create a clear, concise, and patient-friendly summary that explains the key findings in plain language.  This summary will highlight any significant abnormalities and explain their potential implications.  I will also avoid using complex medical jargon and ensure the report is easy to understand.

Please provide the necessary data so I can assist you.
